# Are-MipiA1A2 : Venin

In [1]:
import numpy as np
from numpy import random
import matplotlib.pyplot as plt


#class agent :
#    def __init__(self):
#        self.efficacite= 0.1
#        self.spawn = 0.1
#        self.coagulation = 0.1
#        self.perte = 0.1
#
#globule_rouge = agent()
#globule_blanc = agent()
#plaquette = agent()
#venin = agent()
#coagulation = agent()
#anti_coagulant = agent()



#"""Type : Cell = List[float**6]"""





anti_coagulantefficacite = 0.1
globule_blancefficacite = 0.1
veninefficacite = 0.1
plaquettescoagulation = 0.1
veninechange=0.5
globule_blancechange=0.5

In [2]:
def init_matrice(N):
    """Int -> Mat[Cell]
    crée une matrice de taille N*N de type : Mat[list[Gr,Gb,Pl,Ve,Co,Ac,boost]]"""
    matrix = np.zeros((N,N,7))
    mat = np.random.uniform(low=0.878, high=0.985, size=(N,N))
    for i in range(N):
        for j in range(N):
            x = mat[i,j]
            matrix[i,j]=(x , (1 - x)*0.023 , (1 - x)*0.977,0,(1 - x)*0.977,0,1)
    return matrix
            

In [3]:
def randomlist(N):
    """Int => list[int]*
    retourne une liste de nombres dans un ordre aléatoire len(list)=N """
    list0 = [i for i in range(N)]
    list1=[]
    rlist=[]
    for i in range(n):
        e = np.random.choice(len(list0))
        rlist.append(list0[e])
        list1=[j for j in list0 if j!=list0[e]]
        list0=list1
    return rlist

In [96]:
def selfinteract(mat,N):
    matrix = mat
    a = [0 for i in range(7)]
    for i in range(N):
        for j in range(N):
            
            a = mat[i,j]
            
            #globule_rouge+venin
            if a[3] > 0:
                a[0] = a[0] - a[0]*veninefficacite
            #globule_blanc+venin
            if a[3] > 0:
                a[3] = a[3] - a[1]*globule_blancefficacite
                a[1] = 0
                a[6] = 10
            #plaquettes+plaquettes
            a[4] = a[4] + a[2]*plaquettescoagulation
            a[2] = a[2] - a[2]/plaquettescoagulation
            #coagulation+anti_coagulant
            if a[5]>0:
                a[4] = a[4] - a[5]*anti_coagulantefficacite
                a[5] = a[5] - a[5]/anti_coagulantefficacite
            #globule_blanc+coagulation
            a[4] = a[4] - a[1]*globule_blancefficacite
            #venin+anti_coagulant
            if a[3] > 0 and a[5]:
                a[3] = a[3] - a[5]*anti_coagulantefficacite
                a[5] = a[5] - a[5]/anti_coagulantefficacite
            
            matrix[i,j] = a
    return matrix
            

In [97]:
selfinteract(init_matrice(2),2)

array([[[ 0.95165668,  0.0011119 , -0.4250828 ,  0.        ,
          0.05184338,  0.        ,  1.        ],
        [ 0.9410252 ,  0.00135642, -0.51856538,  0.        ,
          0.06324457,  0.        ,  1.        ]],

       [[ 0.92407055,  0.00174638, -0.66764762,  0.        ,
          0.08142674,  0.        ,  1.        ],
        [ 0.95621118,  0.00100714, -0.38503507,  0.        ,
          0.04695913,  0.        ,  1.        ]]])

In [98]:
def interact(mat,i,j,k,l):
    mat[i,j,3] = mat[i,j,3] + mat[k,l,3]*veninechange
    mat[k,l,3] = mat[k,l,3] + mat[j,k,3]*veninechange
    
    mat[i,j,1] = mat[i,j,1] + mat[k,l,1]*globule_blancechange
    mat[k,l,1] = mat[k,l,1] + mat[i,j,1]*globule_blancechange
    return mat

In [99]:
def inter_1(N,mat):
    
    
    for i in range(N):
        for j in range(N):
            a = mat[i,j]
            if a[3] > 0 :
                k = np.random.choice(4)
                if k == 0 and i > 0:
                    return interact(mat,i,j,i-1,j)
                elif k == 1 and j < N:
                    return interact(mat,i,j,i,j+1)
                elif k == 2 and i < N :
                    return interact(mat,i,j,i+1,j)
                elif k == 3 and j > 0:
                    return interact(mat,i,j,i,j-1)

In [100]:
def main(N,n):
    mat = init_matrice(N)
    rlist=[mat]
    for i in range(n):
        #mat = inter_1(N,mat)
        mat = selfinteract(mat,N)
        rlist.append(mat)
    return lis_co(rlist,N)
            
            


        
    

In [101]:
def lis_co(lis,N):
    """list[mat(cell)]*int => list(mat[float])
    convertie une liste de matrice de cellule en liste de matrice de float (Coagulation)"""
    long=len(lis)
    Lr = np.zeros((long,N,N))
    #mat : mat(cell)
    for n in range(long):
        mat = lis[n]
        for i in range(N):
            for j in range(N):
                Lr[n,i,j] = mat[i,j,4]
                
    return Lr
                

In [102]:
liss = [init_matrice(2) for n in range(4)]

azerty=lis_co(liss,2)
print(azerty)

[[[0.10611133 0.03719163]
  [0.05330873 0.11422311]]

 [[0.10880439 0.04442098]
  [0.07113952 0.07299614]]

 [[0.06501227 0.08950647]
  [0.07450967 0.0510828 ]]

 [[0.02040387 0.06365114]
  [0.10265143 0.07514666]]]


In [103]:
"""def dectohex(n):
    p=8
    hexlist=['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F']
    binlist=[0,0,0,0,0,0,0,0]
    r=''
    while p!=0:
        p=p-1
        if 2**p <= n:
            n=n-2**p
            binlist[p] = 1
    c=0
    s=0
    for i in range(0,4):
        s = s+ (2**i)*binlist[i+4]
        c=c+1
    r = '#'+ hexlist[s] 
    
    c=0
    s=0
    for i in range(0,4):
        s = s+ (2**i)*binlist[i]
        c=c+1
    r = r + hexlist[s] +"0000"
        
    print(binlist)
    return r """

'def dectohex(n):\n    p=8\n    hexlist=[\'0\',\'1\',\'2\',\'3\',\'4\',\'5\',\'6\',\'7\',\'8\',\'9\',\'A\',\'B\',\'C\',\'D\',\'E\',\'F\']\n    binlist=[0,0,0,0,0,0,0,0]\n    r=\'\'\n    while p!=0:\n        p=p-1\n        if 2**p <= n:\n            n=n-2**p\n            binlist[p] = 1\n    c=0\n    s=0\n    for i in range(0,4):\n        s = s+ (2**i)*binlist[i+4]\n        c=c+1\n    r = \'#\'+ hexlist[s] \n    \n    c=0\n    s=0\n    for i in range(0,4):\n        s = s+ (2**i)*binlist[i]\n        c=c+1\n    r = r + hexlist[s] +"0000"\n        \n    print(binlist)\n    return r '

In [104]:
init_matrice(2)

array([[[8.97919838e-01, 2.34784373e-03, 9.97323184e-02, 0.00000000e+00,
         9.97323184e-02, 0.00000000e+00, 1.00000000e+00],
        [9.84956093e-01, 3.46009871e-04, 1.46978976e-02, 0.00000000e+00,
         1.46978976e-02, 0.00000000e+00, 1.00000000e+00]],

       [[9.50740633e-01, 1.13296543e-03, 4.81264012e-02, 0.00000000e+00,
         4.81264012e-02, 0.00000000e+00, 1.00000000e+00],
        [9.03229148e-01, 2.22572959e-03, 9.45451222e-02, 0.00000000e+00,
         9.45451222e-02, 0.00000000e+00, 1.00000000e+00]]])

In [107]:
main(4,0)

array([[[0.10206544, 0.09742526, 0.07442638, 0.052219  ],
        [0.05887162, 0.09633727, 0.10816471, 0.08185485],
        [0.11156716, 0.08742091, 0.03490189, 0.01922098],
        [0.08724011, 0.07318841, 0.11057359, 0.11220134]]])